In [44]:
from pyspark.sql import SparkSession
import pandas as pd
import json
import psycopg2
import redshift_connector
from pyspark.sql import DataFrameWriter
import requests
from os import environ as env


In [3]:
    # Leer las credentials desde un archivo de texto
    credentials = {}
    with open('credentials.txt', 'r') as file:
        for line in file:
            key, value = line.strip().split(': ')
            if key == 'port':
                value = int(value)  # Convertir el valor del puerto a entero
            credentials[key] = value
    
    # Acceder a las credentials individualmente
    usuario = credentials['usuario']
    password = credentials['password']
    host = credentials['host']
    port = credentials['port']
    database = credentials['database']
    api_key = credentials['api_key']


In [9]:
   #Extraigo los datos de las películas con el id y un bucle for
    
movie_list = []

for i in range(1, 500):
    url = f'https://api.themoviedb.org/3/movie/{i}?api_key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        movie_json = response.json()
        movie_id = movie_json['id']
        movie_title = movie_json['title']
        movie_desc = movie_json['overview']
        movie_genre = [genre['name'] for genre in movie_json['genres']]
        movie_release = movie_json['release_date']
        movie_runtime = movie_json['runtime']
        movie_revenue = movie_json['revenue']
        movie_budget = movie_json['budget']
        movie_language = movie_json['original_language']
        movie_popularity = movie_json['popularity']
        movie_list.append({'id': movie_id, 'title': movie_title, 'description': movie_desc, 'genre': movie_genre, 
                           'release_date': movie_release, 'runtime': movie_runtime, 'revenue': movie_revenue, 
                           'budget': movie_budget, 'language': movie_language, 'popularity': movie_popularity})

movies_df = pd.DataFrame(movie_list)

In [10]:
movies_df.head()

,id,title,description,genre,release_date,runtime,revenue,budget,language,popularity
0,2,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,"[Drama, Comedy, Crime]",1988-10-21,73,0,0,fi,9.839
1,3,Shadows in Paradise,"Nikander, a rubbish collector and would-be ent...","[Drama, Comedy, Romance]",1986-10-17,74,0,0,fi,7.793
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[Crime, Comedy]",1995-12-09,98,4257354,4000000,en,21.290
3,6,Judgment Night,"While racing to a boxing match, Frank, Mike, J...","[Action, Crime, Thriller]",1993-10-15,109,12136938,21000000,en,11.782
4,8,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,[Documentary],2006-01-01,80,0,42000,en,2.944


In [11]:
    #no hay nulos, pero lo dejo armado para cuando haya en el proceso ETL
    movies_df = movies_df.dropna() 
    
    #reseteo el indice
    # Paso 1: Restablecer el índice del DataFrame
    movies_df.reset_index(drop=True, inplace=True)
    
    # Paso 2: Ajustar los valores de la columna \id\
    movies_df['id'] = movies_df.index + 1

In [12]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            388 non-null    int64  
 1   title         388 non-null    object 
 2   description   388 non-null    object 
 3   genre         388 non-null    object 
 4   release_date  388 non-null    object 
 5   runtime       388 non-null    int64  
 6   revenue       388 non-null    int64  
 7   budget        388 non-null    int64  
 8   language      388 non-null    object 
 9   popularity    388 non-null    float64
dtypes: float64(1), int64(4), object(5)
memory usage: 30.4+ KB


In [28]:
spark = SparkSession.builder\
                    .appName("Redshift")\
                    .getOrCreate()

In [49]:
DRIVER_PATH = "/home/coder/working_dir/spark_drivers/postgresql-42.5.2.jar"

env['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {DRIVER_PATH} --jars {DRIVER_PATH} pyspark-shell'
env['SPARK_CLASSPATH'] = DRIVER_PATH

# Crear sesión de Spark
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("Spark y Redshift") \
        .config("spark.jars", DRIVER_PATH) \
        .config("spark.executor.extraClassPath", DRIVER_PATH) \
        .getOrCreate()

In [29]:
jdbc_url = f"jdbc:redshift://{host}:{port}/{database}"
connection_properties = {
    "user" : usuario,
    "password" : password,
    "driver" : "com.amazon.redshift.jdbc42.Driver"
}

In [50]:
spark_df = spark.createDataFrame(movies_df)

In [51]:
spark_df.write.format("jdbc").option("url",f"jdbc:redshift://{host}:{port}/{database}").option("dbtable","moviesdf").option("user", usuario).option("password", password).mode("overwrite").save()

Py4JJavaError: An error occurred while calling o104.save.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:299)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:109)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:109)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:47)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.

ERROR: Could not find a version that satisfies the requirement spark-redshift2 (from versions: none)
ERROR: No matching distribution found for spark-redshift2
Note: you may need to restart the kernel to use updated packages.
